# Семинар 24. Генераторы, менеджеры контекста

## Генераторы

- `range`
- `enumerate`
- `map`
- `filter`
- `zip`
- generator expressions, напр. `(x**2 for x in range(10))`


Полезные ссылки:

- [realpython/generators](https://realpython.com/introduction-to-python-generators/)
- [realpython/itertools](https://realpython.com/python-itertools/)

### `yield`

In [33]:
def mygenerator():
    print('Первый вызов')
    yield 1

    print('Второй вызов')
    yield 2

    print('Последний вызов')
    yield 3

In [35]:
for i in mygenerator():
    print(i)

Первый вызов
1
Второй вызов
2
Последний вызов
3


In [37]:
list(mygenerator())

Первый вызов
Второй вызов
Последний вызов


[1, 2, 3]

In [38]:
gen = mygenerator()
next(gen)

Первый вызов


1

In [39]:
next(gen)

Второй вызов


2

In [40]:
next(gen)

Последний вызов


3

### аналог `range`

In [18]:
def sign(x):
    return 1 if x >= 0 else -1

def myrange(start, stop, step=1):
    s = sign(stop - start)
    if s != sign(step):
        return None
    i = start
    while s * i < s * stop:
        yield i
        i += step

In [20]:
list(myrange(0, -5, -1))

[0, -1, -2, -3, -4]

## Задание

- создать аналог генератора `enumerate`

- создать генератор чисел Фибоначчи

- создать аналог генераторов из модуля `itertools`:
    - `cycle(iterable)`
    - `repeat(obj, times=1)`
    - `product(*iterables)`


## Менеджер контекста

```
with context as ctx:
    # do something using ctx
    
# here ctx will be automatically closed
```


Пример:

```
with open('file.txt', 'rt') as f:
    print(f.read())
```


In [61]:
import numpy as np

In [62]:
arr = np.array([0.123456, 1.234])

In [63]:
print(arr)

[0.123456 1.234   ]


In [64]:
with np.printoptions(precision=2):
    print(arr)

[0.12 1.23]


In [63]:
print(arr)

[0.123456 1.234   ]


In [ ]:
import sqlite3

In [44]:
# менеджер контекста для соединения с базой данных sqlite
class DBConn:
 
    def __init__(self, db_name):
        self.db_name = db_name
    
    def __enter__(self):
        self.conn = sqlite3.connect(self.db_name)
        return self.conn
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        self.conn.close()
        if exc_val:
            raise

fname = 'test.db'

In [43]:
with DBConn(fname) as conn:
    cursor = conn.cursor()
    cursor.execute('CREATE TABLE Persons (PersonID int, LastName varchar(255), FirstName varchar(255))')
    conn.commit()

OperationalError: table Persons already exists

In [31]:
with DBConn(fname) as conn:
    cursor = conn.cursor()
    cursor.execute("INSERT INTO Persons VALUES (0, 'Ivan', 'Petrov')")
    conn.commit()

In [32]:
with DBConn(fname) as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM Persons')
    for rec in cursor.fetchall():
        print(rec)

(0, 'Ivan', 'Petrov')


In [54]:
conn = sqlite3.connect(fname)

In [55]:
conn.close()

In [48]:
with sqlite3.connect(fname) as conn:
    cursor = conn.cursor()
    cursor.execute('SELECT * FROM Persons')
    for rec in cursor.fetchall():
        print(rec)

(0, 'Ivan', 'Petrov')


## `contextlib`

- `@contextmanager`
- `contextlib.suppress(*exceptions)`

Полезные ссылки:

- [contextlib](https://docs.python.org/3/library/contextlib.html)

In [41]:
from contextlib import contextmanager

In [57]:
@contextmanager
def open_db(fname):
    try:
        conn = sqlite3.connect(fname)
        yield conn
    finally:
        conn.close()

In [65]:
import os
from contextlib import suppress

In [66]:
os.remove('somefile.tmp')

FileNotFoundError: [WinError 2] The system cannot find the file specified: 'somefile.tmp'

In [67]:
with suppress(FileNotFoundError):
    os.remove('somefile.tmp')

## Задание

- создать менеджер контекста, позволяющий удобно выполнить какие-либо действия в заданной папке (например, создать файл)

- создать менеджер контекста, позволяющий перенаправить `stdout` в другой поток (например, в файл)
    - необходимо временно связать ссылку `sys.stdout` с другим потоком
